In [1]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import joblib
import matplotlib.pyplot as plt


2025-10-15 10:19:16.639683: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
LATITUDE = 16.0471       # ví dụ: Đà Nẵng
LONGITUDE = 108.2068
MODEL_PATH = "hybrid_cnn_lstm_weather.keras"  # mô hình bạn đã train
SCALER_PATH = "scaler.joblib"                 # scaler đã lưu khi train
INPUT_LEN = 24 * 7       # 7 ngày đầu vào
OUTPUT_LEN = 24          # model dự báo 24 giờ tới (1 ngày)
FEATURES = [
    "T2M","QV2M","PS","WS10M",
    "PRECTOTCORR","CLRSKY_SFC_SW_DWN",
    "hour","day","month"
]


In [3]:
def get_nasa_power_hourly(lat, lon, start_date, end_date):
    params = ["T2M","QV2M","PS","WS10M","PRECTOTCORR","CLRSKY_SFC_SW_DWN"]
    url = (
        f"https://power.larc.nasa.gov/api/temporal/hourly/point?"
        f"parameters={','.join(params)}&community=RE"
        f"&longitude={lon}&latitude={lat}"
        f"&start={start_date.strftime('%Y%m%d')}"
        f"&end={end_date.strftime('%Y%m%d')}"
        f"&format=JSON"
    )
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()["properties"]["parameter"]

    timestamps = [pd.to_datetime(t, format="%Y%m%d%H") for t in data["T2M"].keys()]
    df = pd.DataFrame({"date": timestamps})
    for p in params:
        df[p] = list(data[p].values())

    # Thêm đặc trưng phụ trợ
    df["hour"] = df["date"].dt.hour
    df["day"] = df["date"].dt.day
    df["month"] = df["date"].dt.month
    df["Latitude"] = lat
    df["Longitude"] = lon
    return df

In [ ]:
def preprocess_features(df, scaler=None):
    df = df.copy()
    if scaler is None:
        scaler = MinMaxScaler()
        df[FEATURES] = scaler.fit_transform(df[FEATURES])
    else:
        df[FEATURES] = scaler.transform(df[FEATURES])
    return df, scaler

In [4]:
def make_input_sequence(df):
    arr = df[FEATURES].values.astype("float32")
    X = arr[-INPUT_LEN:]              # lấy 7 ngày gần nhất
    return np.expand_dims(X, axis=0)  # (1, INPUT_LEN, 9)

In [5]:
def forecast_T2M():
    print("📡 Đang tải dữ liệu NASA POWER...")
    end_date = dt.datetime.utcnow()
    start_date = end_date - dt.timedelta(days=45)
    df = get_nasa_power_hourly(LATITUDE, LONGITUDE, start_date, end_date)
    print("✅ Dữ liệu NASA:", df.shape)

    # Load scaler
    try:
        scaler = joblib.load(SCALER_PATH)
        print("📦 Đã load scaler huấn luyện.")
    except:
        print("⚠️ Không tìm thấy scaler, sẽ fit tạm thời.")
        scaler = MinMaxScaler()
        scaler.fit(df[FEATURES])

    # Scale & tạo input
    df_scaled, _ = preprocess_features(df, scaler)
    X_input = make_input_sequence(df_scaled)

    # Load model
    print("🧠 Đang tải mô hình:", MODEL_PATH)
    model = keras.models.load_model(MODEL_PATH)

    # Dự báo
    print("🤖 Đang dự báo 24 giờ tới (T2M)...")
    y_pred = model.predict(X_input)[0]  # (24,)
    dummy = np.zeros((len(y_pred), len(FEATURES)))
    dummy[:, 0] = y_pred                # chỉ cột T2M
    y_pred_inverse = scaler.inverse_transform(dummy)[:, 0]

    # Tạo thời gian tương lai
    last_date = df["date"].iloc[-1]
    future_dates = [last_date + dt.timedelta(hours=i+1) for i in range(len(y_pred_inverse))]

    # DataFrame kết quả
    pred_df = pd.DataFrame({
        "date": future_dates,
        "T2M_predicted": y_pred_inverse
    })

    print("\n📊 Kết quả dự báo nhiệt độ (T2M):")
    print(pred_df.head(24))

    # Vẽ biểu đồ
    plt.figure(figsize=(10,5))
    plt.plot(pred_df["date"], pred_df["T2M_predicted"], label="Dự báo T2M (°C)")
    plt.title("🌤 Dự báo Nhiệt độ 24 giờ tới")
    plt.xlabel("Thời gian")
    plt.ylabel("Nhiệt độ (°C)")
    plt.legend()
    plt.show()

    # Lưu kết quả
    pred_df.to_csv("forecast_T2M_result.csv", index=False)
    print("💾 Đã lưu kết quả -> forecast_T2M_result.csv")

In [6]:
if __name__ == "__main__":
    forecast_T2M()

📡 Đang tải dữ liệu NASA POWER...


/tmp/ipykernel_7364/3152177213.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_date = dt.datetime.utcnow()


✅ Dữ liệu NASA: (1104, 12)
📦 Đã load scaler huấn luyện.


NameError: name 'preprocess_features' is not defined